In [1]:
import pandas as pd
import numpy as np

In [14]:
HED_df = pd.read_excel('Meter 1 - Head End Read Data.xlsx', sheet_name='Meter Reads HES')
MDMS_df = pd.read_excel('Meter 1 - MDMS Read Data.xlsx', sheet_name='MDMS Meter reads')

In [15]:
HED_df.head()

,Intervals (UTC),Analysis,Interval Status,Consumption (Liters) - Status,Consumption (Liters) - Usage,Consumption (Liters) - Reading
0,2025-11-06 16:00:00,NaN,NaN,--,0,0
1,2025-11-06 17:00:00,Potential Fill,NaN,--,--,--
2,2025-11-06 18:00:00,Potential Fill,NaN,--,--,--
3,2025-11-06 19:00:00,Potential Fill,NaN,--,--,--
4,2025-11-06 20:00:00,Potential Fill,NaN,--,--,--


In [16]:
MDMS_df.head()

,Meter Serial Number,Channel Name,Read End Time,Read Value,Validation Status,Flags,Fail Code,Type,Status
0,Meter 1,Registered,2025-11-06 01:00:00,0.000000,VAL,0.0,NaN,Register,Active
1,Meter 1,Registered,2025-11-07 00:00:00,306.151261,EST,NaN,NaN,Register,Active
2,Meter 1,Registered,2025-11-08 00:00:00,625.613445,EST,NaN,NaN,Register,Active
3,Meter 1,Registered,2025-11-09 00:00:00,945.075630,EST,NaN,NaN,Register,Active
4,Meter 1,Registered,2025-11-10 00:00:00,1264.537815,EST,NaN,NaN,Register,Active
